In [2]:
# 计算深度学习特征
# 参考https://github.com/AIM-Harvard/foundation-cancer-image-biomarker.git
import pandas as pd
import os
import SimpleITK as sitk

dataset_list = ["tjh"]

for dataset in dataset_list:
    data_path = f'./dataset/{dataset}/crop_image'
    mask_path = f'./dataset/{dataset}/crop_label'
    
    # 创建列表存储所有行数据
    rows = []

    for name in os.listdir(data_path):
        image_path = os.path.join(data_path, name)
        mask_file = os.path.join(mask_path, name)

        try:
            # 读取mask文件
            mask = sitk.ReadImage(mask_file)

            # 获取mask的中心点
            label_shape_filter = sitk.LabelShapeStatisticsImageFilter()
            label_shape_filter.Execute(mask)

            # 确保标签2存在
            if label_shape_filter.HasLabel(2):
                centroid = label_shape_filter.GetCentroid(2)
                x, y, z = centroid

                # 创建行数据
                row = {
                    "image_path": image_path,
                    "coordX": x,
                    "coordY": y,
                    "coordZ": z
                }
                rows.append(row)
            else:
                print(f"Warning: Label 2 not found in {name}")

        except Exception as e:
            print(f"Error processing {name}: {str(e)}")

    # 一次性创建DataFrame
    if rows:
        df = pd.DataFrame(rows)
        output_path = f'./coord_{dataset}.csv'
        df.to_csv(output_path, index=False)
        print(f"Successfully created {output_path}")
    else:
        print(f"No valid data found for dataset {dataset}")

Successfully created ./coord_tjh.csv


In [4]:
from monai.networks.nets import resnet50 as resnet50_monai
import torch
model = resnet50_monai(pretrained=False, n_input_channels=1, widen_factor=2, conv1_t_stride=2, feed_forward=False)
model.load_state_dict(torch.load('./foundation-cancer-image-biomarker/fmcib.torch')['trunk_state_dict'])
device = torch.device('cuda:0')
model.to(device)
model.eval()

ResNet(
  (conv1): Conv3d(1, 128, kernel_size=(7, 7, 7), stride=(2, 2, 2), padding=(3, 3, 3), bias=False)
  (bn1): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool3d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): ResNetBottleneck(
      (conv1): Conv3d(128, 128, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
      (bn1): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv3d(128, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (bn2): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv3d(128, 512, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
      (bn3): BatchNorm3d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Co

In [14]:
import ssl
import urllib.request

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
from fmcib.run import get_features

feature_df = get_features("./coord_tjh.csv")
feature_df.to_csv("./dl_tjh.csv", index=False)

2024-12-23 20:24:26.707 | INFO     | fmcib.run:get_features:18 - Loading CSV file ...
2024-12-23 20:24:26.715 | INFO     | fmcib.preprocessing:get_dataloader:50 - Building dataloader instance ...
2024-12-23 20:24:26.724 | INFO     | fmcib.models.resnet50:resnet50:19 - Loading pretrained foundation model (Resnet50) on cuda...


URLError: <urlopen error [Errno 101] Network is unreachable>

In [ ]:
import SimpleITK as sitk
from fmcib.utils import download_LUNG1, build_image_seed_dict
from fmcib.visualization import visualize_seed_point